# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [50]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

def scrape_imdb(title_type, user_rating, start_date, end_date):
    url = url = f"https://www.imdb.com/search/title/?title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating},10"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    movies = []

    for div in soup.find_all('div', class_='lister-item mode-advanced'):
        nr = div.find('span', class_='lister-item-index unbold text-primary').text.strip('.').strip()
        title = div.find('h3').find('a').text.strip()
        year = div.find('span', class_='lister-item-year').text.strip('()')
        rating = div.find('strong').text
        runtime = div.find('span', class_='runtime').text.strip(' min') if div.find('span', class_='runtime') else None
        genre = div.find('span', class_='genre').text.strip() if div.find('span', class_='genre') else None
        description = div.find_all('p', class_='text-muted')[1].text.strip()
        director_and_stars = div.find('p', class_='').find_all('a')
        director = director_and_stars[0].text if director_and_stars else 'N/A'
        stars = ', '.join([a.text for a in director_and_stars[1:]]) if director_and_stars else 'N/A'
        votes = div.find('span', attrs={'name': 'nv'})['data-value'] if div.find('span', attrs={'name': 'nv'}) else None
        gross = div.find_all('span', attrs={'name': 'nv'})[1]['data-value'] if len(div.find_all('span', attrs={'name': 'nv'})) > 1 else None

        movie_details = {'Movie Nr': nr, 'Title': title, 'Year': year, 'Rating': rating, 'Runtime (min)': runtime, 'Genre': genre, 'Description': description, 'Director': director, 'Stars': stars, 'Votes': votes, 'Gross ($M)': gross,}
        movies.append(movie_details)

    df = pd.DataFrame(movies)
    return df

df = scrape_imdb(title_type='feature', user_rating='7.5', start_date='1990-01-01', end_date='1992-12-31')
df

""


In [54]:
#Bonus 
import pandas as pd
from bs4 import BeautifulSoup
import requests

def scrape_imdb(title_type, user_rating, start_date, end_date):
    url = f"https://www.imdb.com/search/title/?title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating},10"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    total_movies = int(soup.find('div', class_='desc').find('span').text.split('of')[-1].split('titles')[0].strip())

    pages = total_movies // 50 + (1 if total_movies % 50 != 0 else 0)
    movies = []
    for page in range(1, pages * 50, 50):
        url_page = url + f'&start={page}&ref_=adv_nxt'
        response_page = requests.get(url_page)
        soup_page = BeautifulSoup(response_page.content, 'html.parser')
   
        for div in soup_page.find_all('div', class_='lister-item mode-advanced'):
            nr = div.find('span', class_='lister-item-index unbold text-primary').text.strip('.').strip()
            title = div.find('h3').find('a').text.strip()
            year = div.find('span', class_='lister-item-year text-muted unbold').text.strip('()')
            rating = div.find('strong').text
            runtime = div.find('span', class_='runtime').text.strip(' min') if div.find('span', class_='runtime') else None
            genre = div.find('span', class_='genre').text.strip() if div.find('span', class_='genre') else None
            description = div.find_all('p', class_='text-muted')[1].text.strip()
            director_and_stars = div.find('p', class_='').find_all('a')
            director = director_and_stars[0].text if director_and_stars else 'N/A'
            stars = ', '.join([a.text for a in director_and_stars[1:]]) if director_and_stars else 'N/A'
            votes = div.find('span', attrs={'name': 'nv'})['data-value'] if div.find('span', attrs={'name': 'nv'}) else None
            gross = div.find_all('span', attrs={'name': 'nv'})[1]['data-value'] if len(div.find_all('span', attrs={'name': 'nv'})) > 1 else None

        movie_details = {'Movie Nr': nr, 'Title': title, 'Year': year, 'Rating': rating, 'Runtime (min)': runtime, "Genre": genre, 'Description': description, 'Director': director, 'Stars': stars, 'Votes': votes, 'Gross ($M)': gross}

        df = pd.DataFrame(movies)
        return df
    

    df = scrape_imdb(title_type='feature', user_rating='7.5', start_date='1990-01-01', end_date='1992-12-31')
    df